In [3]:
import pandas as pd
import numpy as np
import dataImport as dataI

In [27]:
train = pd.read_csv('../Data/youtube.csv')
# n_columns = len(prueba.columns)
# print columns_cant

# data_dict ={}

# for column in prueba.columns:
#     data_dict[column] = prueba.ix[:, column].tolist()
    
# print data_dict['windy']

train.head(20)

,Outlook,temps,humedity,windy,hours
0,1,3,1,0,25
1,1,3,1,1,30
2,2,3,1,0,46
3,3,2,1,0,45
4,3,1,0,0,52
5,3,1,0,1,23
6,2,1,0,1,43
7,1,2,1,0,35
8,1,1,0,0,38
9,3,2,0,0,46


In [32]:
train.temps.unique()
print train.loc[train['temps']==3,'hours'].values.flatten()
print  train.loc[train['temps']==3,'hours'].count()
print train['temps'].count()



[25 30 46 44]
4
14


In [41]:
def std_value(df,column,value,target):
    std = np.std(df.loc[train[column]==value,target].values.flatten())
    len = df[column].count()
    len_value = df.loc[train[column]==value,target].count()
    prob = float(len_value)/len
    return std*prob

In [42]:
def calculate_std(df,column,target):
    stds = 0
    values_uniques = df[column].unique()
    for value in values_uniques:
        stds +=  std_value(df,column,value,target)
    return stds
    

In [43]:
def calculate_stds(df,columns,target):
    stds=[]
    for column in columns:
        stds.append((column,calculate_std(df,column,target)))
    return stds
        

In [47]:
def build_tree(df,target):  # target = nombre de la columna a predecir
    # Calculo de la std de la variable a predecir
    target_std = np.std(df.ix[:, target].tolist())
    
    # calculamos la desviacion standard de cada columna:
    # Esto es: la sumatoria de (la probabilidad de cada uno de los distintos valores
    # de una columna * la desviacion standar del mismo) 
    stds_columns = calculate_stds(df,['Outlook','temps','humedity','windy'],target)
    
    stds_columns(lambda x: target_std - x[1])
    
    print stds_columns
    
    
    
    
build_tree(train,'hours')

TypeError: 'list' object is not callable